In [28]:
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf

import imageio as img

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu

In [56]:
# setting variables for the directory to the data
todata = "/home/abdullah/Desktop/Abdullah/LUMS/Senior/Sproj/ImgPreProcessing/Channel/"
moredata = "/home/abdullah/Desktop/Abdullah/LUMS/Senior/Sproj/ImgPreProcessing/AmplifiedDataset"


training_iters = 200 
learning_rate = 0.001 
batch_size = 128

# data input (img shape: 28*28)
n_input = 28

# total classes (0-9 digits)
n_classes = 26

In [64]:
# saving current directory
cur = os.getcwd()

# changing directoy to data set
os.chdir(todata)
# getting all the folder names
nmes = os.listdir(".")
nmes.sort()

# reading all the data into labels and data numpy arrays
labels = []
data = []
for letr in nmes:
    for file in os.listdir(todata+letr):
        labels.append(letr)
        f = img.imread(todata + letr + "/"+ file)
#         print(f.dtype, f.shape, type(f))
        data.append(f)
#         break
#     break

print("\nData labales matrix")
labels = np.array(labels)
print(labels, labels.dtype, labels.shape)

# diving by 255 to get clamp values between zero and one
print("\nData matrix")
data = np.array(data, np.float32)/255
print(data.dtype, data.shape)



Data labales matrix
['A' 'A' 'A' ... 'Z' 'Z' 'Z'] <U1 (3844,)

Data matrix
float32 (3844, 28, 28)


In [57]:
# converting labels into onehot encodings

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
# print(integer_encoded)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded.shape)

(3844, 26)


/home/abdullah/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [84]:
# splitting up the data
tr, tr_l, vl, vl_l = train_val_split(data, onehot_encoded, 0.7)



The splits are: 
(2690, 28, 28)
(2690, 26)
(1154, 28, 28)
(1154, 26)


In [85]:
# Reshape training and testing image
train_X = tr.reshape(-1, n_input, n_input, 1)
test_X = vl.reshape(-1,n_input,n_input,1)



train_y = tr_l
test_y = vl_l

train_X.shape, test_X.shape, train_y.shape, test_y.shape

((2690, 28, 28, 1), (1154, 28, 28, 1), (2690, 26), (1154, 26))

In [83]:
# function that splits up the data

def train_val_split(data, labels, keep):
  

    data = np.array(data)
    labels = np.array(labels)

    #split the data
    number_of_validation_points = data.shape[0]

    split = int(number_of_validation_points*keep)

    # getting curent state so shuffle is the same for both arrays
    curState = np.random.get_state()
    np.random.shuffle(data)
    # setting the state
    np.random.set_state(curState)
    np.random.shuffle(labels)



    #images are unflattened
    temp_val = data[split:]
    val_label = labels[split:]
    temp_train = data[:split]
    train_label = labels[:split]

    val = temp_val
    train = temp_train
    
#     #flatten the images
#     val = []
#     train = []

#     for element in temp_val:
#         val.append(element.flatten())

#     for element in temp_train:
#         train.append(element.flatten())


    train = np.array(train)
    val = np.array(val)

    print("The splits are: ")
    print(train.shape)   
    print(train_label.shape)
    print(val.shape) 
    print(val_label.shape)

    return train.T, train_label, val.T, val_label



In [ ]:
# the CNN begins



In [ ]:
# input dimen and output dimen
x = tf.placeholder("float", [None, n_input,n_input,1])
y = tf.placeholder("float", [None, n_classes])

In [ ]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x) 

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')

In [ ]:
weights = {
    'wc1': tf.get_variable('W0', shape=(3,3,1,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc2': tf.get_variable('W1', shape=(3,3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc3': tf.get_variable('W2', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('W3', shape=(4*4*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W6', shape=(128,n_classes), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(10), initializer=tf.contrib.layers.xavier_initializer()),
}

In [ ]:
def conv_net(x, weights, biases):  

    # here we call the conv2d function we had defined above and pass the input image x, weights wc1 and bias bc1.
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 14*14 matrix.
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    # here we call the conv2d function we had defined above and pass the input image x, weights wc2 and bias bc2.
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 7*7 matrix.
    conv2 = maxpool2d(conv2, k=2)

    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 4*4.
    conv3 = maxpool2d(conv3, k=2)


    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
pred = conv_net(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
#Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

#calculate accuracy across all the given images and average them out. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init) 
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    for i in range(training_iters):
        for batch in range(len(train_X)//batch_size):
            batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
            batch_y = train_y[batch*batch_size:min((batch+1)*batch_size,len(train_y))]    
            # Run optimization op (backprop).
                # Calculate batch loss and accuracy
            opt = sess.run(optimizer, feed_dict={x: batch_x,
                                                              y: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y})
        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
        print("Optimization Finished!")

        # Calculate accuracy for all 10000 mnist test images
        test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: test_X,y : test_y})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()